In [1]:
import pandas as pd
import numpy as np


In [2]:
original_data = pd.read_csv('Transformed_Minedex_data.csv')
original_data.head()

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne),ppm
0,S0000001,13/07/2005,14/09/2006,0.695,1.30,gram/tonne,Gold,903.50,kilogram,NaN,GOLD,0.90350,1.30
1,S0000002,12/03/2019,NaN,0.528,2.09,gram/tonne,Gold,1103.52,kilogram,0.3,GOLD,1.10352,2.09
2,S0000005,30/09/2006,07/03/2016,0.011,7.33,gram/tonne,Gold,80.63,kilogram,NaN,GOLD,0.08063,7.33
3,S0000005,30/09/2006,07/03/2016,0.018,4.94,gram/tonne,Gold,88.92,kilogram,NaN,GOLD,0.08892,4.94
4,S0000006,08/10/1993,30/12/2014,0.057,2.36,gram/tonne,Gold,134.52,kilogram,NaN,GOLD,0.13452,2.36


In [3]:
original_data.columns

Index(['SiteCode', 'StartDate', 'EndDate', 'Tonnage (Mt)', 'Grade',
       'GradeUnit', 'CommodityName', 'ContainedMetal', 'ContainedMetalUnit',
       'CutOffGrade', 'TargetCommodityGroup', 'quantity(tonne)', 'ppm'],
      dtype='object')

In [4]:
df=original_data[original_data['CommodityName'] == 'Nickel']

In [8]:
df2 = original_data[original_data['SiteCode'].isin(list(df.SiteCode.unique()))]

In [9]:
df2.CommodityName.unique()

array(['Palladium', 'Gold', 'Platinum', 'Rhodium', 'Scandium', 'Silver',
       'Platinum group elements', 'Nickel', 'Arsenic', 'Carbonate',
       'Magnesium', 'Aluminium', 'Zinc', 'Calcium', 'Copper', 'Iron',
       'Cobalt', 'Iron - ferric oxide', 'Sulfur', 'Magnesium oxide',
       'Silica', 'Manganese', 'Chromium', 'Alumina', 'Lime'], dtype=object)

In [10]:
def filtered_data(df):

    site_minerals=[]
    for i in range(len(df.SiteCode.unique())):
        a=df[df['SiteCode'] == df.SiteCode.unique()[i]]
        g = a.groupby('CommodityName')['ppm'].sum()
        g=dict(g)
        site_minerals.append(g)
        
    row=[]
    for site in site_minerals:
        dic={}
        for i in df.CommodityName.unique():
            if i in site.keys():
                dic[i]=site[i]
            else:
                dic[i]=0
        row.append(dic)

    new_data= pd.DataFrame(row)
    new_data.insert(0, "SiteCode",  list(df.SiteCode.unique()), True)
    
    return new_data

In [11]:
# Create table of commodites for each site code (the quantities are summed up)
test_data=filtered_data(df2)

In [14]:
# Filter dataset by minerals appear in WAMEX data
test_data.head()

,SiteCode,Nickel,Copper,Cobalt,Palladium,Gold,Platinum,Rhodium,Iron,Sulfur,Silver
0,S0001805,15100.0,4200.0,0.0,11.83,1.48,11.43,0.0,0.0,0.0,0.0
1,S0001806,2700.0,4600.0,0.0,4.60,0.80,3.30,0.3,0.0,0.0,0.0
2,S0004096,1800.0,3200.0,0.0,0.74,0.22,0.89,0.0,0.0,0.0,0.0
3,S0005584,27100.0,38100.0,1400.0,2.49,0.00,0.00,0.0,0.0,0.0,0.0
4,S0016715,12500.0,0.0,620.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0


In [15]:
test_data=test_data.loc[:, ['SiteCode','Nickel','Copper','Cobalt',"Palladium",'Gold','Platinum','Rhodium','Iron','Sulfur','Silver']] 

In [16]:
Sites_data = pd.read_csv('Sites.csv')

In [17]:
Sites_data=Sites_data.loc[:,['Latitude','Longitude','SiteCode']]

In [18]:
# Add Latitude, Longitude to dataframe
test_data=pd.merge(test_data,Sites_data,on='SiteCode',how='left')

In [19]:
test_data.head()

,SiteCode,Nickel,Copper,Cobalt,Palladium,Gold,Platinum,Rhodium,Iron,Sulfur,Silver,Latitude,Longitude
0,S0001805,15100.0,4200.0,0.0,11.83,1.48,11.43,0.0,0.0,0.0,0.0,-17.751530,127.831596
1,S0001806,2700.0,4600.0,0.0,4.60,0.80,3.30,0.3,0.0,0.0,0.0,-21.120370,116.828769
2,S0004096,1800.0,3200.0,0.0,0.74,0.22,0.89,0.0,0.0,0.0,0.0,-26.845022,117.739164
3,S0005584,27100.0,38100.0,1400.0,2.49,0.00,0.00,0.0,0.0,0.0,0.0,-20.985479,116.866849
4,S0016715,12500.0,0.0,620.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,-30.020500,121.149002


In [21]:
test_data.describe()

,Nickel,Copper,Cobalt,Palladium,Gold,Platinum,Rhodium,Iron,Sulfur,Silver,Latitude,Longitude
count,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000,346.000000
mean,32469.508665,1201.157920,447.901734,0.087439,0.016734,0.068792,0.001734,23665.872877,770.520231,0.037630,-29.361858,121.354089
std,41370.124824,4236.208925,751.035831,0.747362,0.134488,0.671506,0.022775,111638.295212,6979.148926,0.317268,3.273697,1.871747
min,1800.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-33.666733,116.387001
25%,10225.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-31.500785,120.573832
50%,17200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-30.189752,121.639933
75%,37000.000000,0.000000,700.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.145195,121.858426
max,371000.000000,38100.000000,4400.000000,11.830000,1.630000,11.430000,0.300000,861000.000000,107000.000000,3.200000,-17.294567,128.186048


## Train random forrest model on WAMEX data for unique commotities



In [22]:
train_data = pd.read_csv('Merged_Ni_lessthan15km.csv',index_col=0)
train_data.head()

,Unnamed: 0_x,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag,lat,long,Unnamed: 0_y,minedex_coords,minedex_site,wamex_lat,wamex_lon,distance
0,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46661,"(-17.34893, 128.022095)",S0001442,-17.234642,128.032146,12.693593881734987 km
1,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46662,"(-17.31078, 128.023499)",S0002890,-17.234642,128.032146,8.476376684524638 km
2,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46663,"(-17.3526, 128.027695)",S0019720,-17.234642,128.032146,13.063277176872425 km
3,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46664,"(-17.3526, 128.027695)",S0019821,-17.234642,128.032146,13.063277176872425 km
4,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46665,"(-17.338411, 128.017007)",S0231765,-17.234642,128.032146,11.59661465949521 km


In [23]:
# Using Eli's code for precessing WAMEX dataset
df_unique = train_data.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'lat', 'long','minedex_coords', 'minedex_site','distance'])
df_unique = df_unique.drop_duplicates()

### Transform data using log transform

In [24]:
train_data = df_unique.fillna(0)

In [25]:
#Seperate dataset (exclude long and lat)
train_data=train_data.iloc[:,:-2]

In [26]:
train_data=train_data.replace(0,1)


In [27]:
train_data

,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag
0,56.0,420.0,9.0,14.0,0.010,1.0,1.0,1.0,1.000000,1.0
5,31.0,195.0,6.0,10.0,0.007,1.0,1.0,1.0,1.000000,1.0
10,90.0,150.0,15.0,2.0,0.004,0.5,1.0,1.0,1.000000,1.0
15,37.0,135.0,11.0,4.0,0.005,1.0,1.0,1.0,1.000000,1.0
18,58.0,120.0,14.0,5.0,0.004,0.5,1.0,1.0,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...
150552,1300.0,45.0,105.0,1.0,1.000,1.0,1.0,1.0,0.000024,1.0
150561,1950.0,120.0,690.0,10.0,1.000,8.0,1.0,1.0,0.000037,1.0
150570,530.0,175.0,240.0,14.0,1.000,14.0,1.0,1.0,0.000033,1.0
150579,1150.0,10.0,45.0,0.5,1.000,3.0,0.5,4500.0,1.000000,1.0


In [28]:
train_data = train_data.apply(np.log, axis=1)


In [29]:
train_data['wamex_lat'] = df_unique['wamex_lat']
train_data['wamex_lon'] = df_unique['wamex_lon']

In [30]:
train_data.head()

,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag,wamex_lat,wamex_lon
0,4.025352,6.040255,2.197225,2.639057,-4.605170,0.000000,0.0,0.0,0.0,0.0,-17.234642,128.032146
5,3.433987,5.273000,1.791759,2.302585,-4.961845,0.000000,0.0,0.0,0.0,0.0,-17.234642,128.032146
10,4.499810,5.010635,2.708050,0.693147,-5.521461,-0.693147,0.0,0.0,0.0,0.0,-17.234642,128.032146
15,3.610918,4.905275,2.397895,1.386294,-5.298317,0.000000,0.0,0.0,0.0,0.0,-17.216592,128.047184
18,4.060443,4.787492,2.639057,1.609438,-5.521461,-0.693147,0.0,0.0,0.0,0.0,-17.216028,128.048165


In [47]:
y = train_data['Co']
X = train_data.drop(columns=['Co'])

In [48]:
X.describe()

,Ni,Cu,Pd,Au,Pt,Rh,Fe,S,Ag,wamex_lat,wamex_lon
count,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000,23233.000000
mean,4.846256,4.120914,0.512123,-3.076138,0.699706,0.035900,5.673807,-1.825698,-0.967564,-27.805376,122.064656
std,2.051320,1.609004,1.345138,3.835590,1.576857,0.433290,4.503900,2.493113,2.826437,4.027483,3.081927
min,-7.600902,-7.600902,-6.214608,-14.508658,-5.115996,-0.693147,-2.501036,-17.727534,-12.206073,-33.697944,116.266019
25%,3.465736,3.280911,0.000000,-6.214608,0.000000,0.000000,0.000000,-3.506558,-1.832581,-30.972928,120.517366
50%,4.605170,4.001864,0.000000,-4.605170,0.000000,0.000000,8.160518,0.000000,0.000000,-28.094057,121.117641
75%,6.413459,4.976734,0.693147,0.000000,0.916291,0.000000,9.342245,0.000000,0.000000,-26.123169,122.515106
max,13.304685,13.554146,15.424848,8.149024,12.296827,9.210340,13.588610,4.605170,11.701063,-17.187316,128.993500


### Train the regression model

In [49]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [50]:
#Import Random Forest Regressor module
from sklearn.ensemble import RandomForestRegressor
# Create a based model
rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [51]:
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)

from sklearn.metrics import mean_squared_error

print(f"Train MSE: {mean_squared_error(train_preds, y_train)}")
print(f"Test MSE: {mean_squared_error(test_preds, y_test)}")

Train MSE: 0.05701437367659089
Test MSE: 0.38465145481210183


### Apply trained model for the Minedex data

In [52]:
original_test_data=test_data
data_pred_Co=test_data.loc[:, ['Nickel','Copper',"Palladium",'Gold','Platinum','Rhodium','Iron','Sulfur','Silver','Latitude','Longitude']] 

In [53]:
#Seperate dataset (exclude long and lat)
data_pred_Co=data_pred_Co.iloc[:,:-2]

In [54]:
data_pred_Co=data_pred_Co.replace(0,1)

In [55]:
data_pred_Co = data_pred_Co.apply(np.log, axis=1)

In [56]:
data_pred_Co['Latitude'] = original_test_data['Latitude']
data_pred_Co['Longitude'] = original_test_data['Longitude']

In [57]:
data_pred_Co.head()

,Nickel,Copper,Palladium,Gold,Platinum,Rhodium,Iron,Sulfur,Silver,Latitude,Longitude
0,9.622450,8.342840,2.470639,0.392042,2.436241,0.000000,0.0,0.0,0.0,-17.751530,127.831596
1,7.901007,8.433812,1.526056,-0.223144,1.193922,-1.203973,0.0,0.0,0.0,-21.120370,116.828769
2,7.495542,8.070906,-0.301105,-1.514128,-0.116534,0.000000,0.0,0.0,0.0,-26.845022,117.739164
3,10.207289,10.547970,0.912283,0.000000,0.000000,0.000000,0.0,0.0,0.0,-20.985479,116.866849
4,9.433484,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-30.020500,121.149002
...,...,...,...,...,...,...,...,...,...,...,...
341,9.296518,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-32.095677,121.644431
342,8.411833,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-32.201621,121.664588
343,9.546813,9.104980,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-31.591497,123.441046
344,9.546813,9.104980,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-31.591497,123.441046


In [58]:
test_preds = np.exp(rf.predict(data_pred_Co))

In [59]:
result=original_test_data.loc[:,['SiteCode','Cobalt']]

In [60]:
result['Pred_Cobalt'] = test_preds


In [61]:
result

,SiteCode,Cobalt,Pred_Cobalt
0,S0001805,0.0,661.232620
1,S0001806,0.0,34.963049
2,S0004096,0.0,71.950038
3,S0005584,1400.0,723.192013
4,S0016715,620.0,397.891845
...,...,...,...
341,S0236620,2300.0,11746.647845
342,S0236621,1100.0,1462.703119
343,S0236838,700.0,698.475479
344,S0236838,700.0,698.475479
